In [1]:
import requests
import re

from lxml import html
from bs4 import BeautifulSoup
import random

In [2]:
class Lexico:
    def __init__(self, id_lex, nome, nocao, classificacao, impacto, sinonimo, relacionados):
        self.id_lex = int(id_lex)
        self.nome = str(nome)
        self.nocao = str(nocao)
        self.classificacao = str(classificacao)
        self.impacto = str(impacto)
        self.sinonimo = str(sinonimo)
        self.relacionados = list(relacionados)

In [3]:
class Cenario:
    def __init__(self, id_cen, titulo, objetivo, contexto, atores, recursos, excecao, episodios, cen_relacionados):
        self.id_cen = int(id_cen)
        self.titulo = str(titulo)
        self.objetivo = str(objetivo)
        self.contexto = str(contexto)
        self.atores = str(atores)
        self.recursos = recursos
        self.excecao = excecao
        self.episodios = episodios
        self.cen_relacionados = cen_relacionados

In [4]:
urls_lexicos = [
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12529&t=l', 12529], # administrador
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12502&t=l', 12502], # amigo
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12534&t=l', 12534], # banido
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12574&t=l', 12574], # bloquear usuário
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12506&t=l', 12506], # comentário
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12525&t=l', 12525], # comunidade
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12533&t=l', 12533], # denunciar
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12496&t=l', 12496], # desafios
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12495&t=l', 12495], # emblemas
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12518&t=l', 12518], # emblemas conquistados
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12516&t=l', 12516], # emblemas mais próximos
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12517&t=l', 12517], # emblemas recentes
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12531&t=l', 12531], # infrator
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12499&t=l', 12499], # matéria
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12490&t=l', 12490], # melhor resposta
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12508&t=l', 12508], # mensagens
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12528&t=l', 12528], # moderador
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12491&t=l', 12491], # nível
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12509&t=l', 12509], # notificações
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12497&t=l', 12497], # obrigados
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12532&t=l', 12532], # perfil
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12494&t=l', 12494], # pergunta
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12510&t=l', 12510], # pergunta expirada
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12575&t=l', 12575], # pergunta popular
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12492&t=l', 12492], # pontos
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12498&t=l', 12498], # rankings
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12520&t=l', 12520], # recebendo resposta
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12489&t=l', 12489], # responder
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12521&t=l', 12521], # respondidas
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12522&t=l', 12522], # respostas verificadas
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12519&t=l', 12519], # sem resposta
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12514&t=l', 12514], # tarefas adicionadas
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12512&t=l', 12512], # tarefas resolvidas
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12576&t=l', 12576], # time de respostas
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=12577&t=l', 12577]  # universitário
]

In [5]:
urls_cenarios = [
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=2999&t=c', 2999],
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=2996&t=c', 2996],
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=2998&t=c', 2998],
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=3030&t=c', 3030],
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=3000&t=c', 3000],
    ['http://pes.inf.puc-rio.br/cel/aplicacao/main.php?id=2997&t=c', 2997]
]

In [6]:
session_requests = requests.session()
PAYLOAD = 'http://pes.inf.puc-rio.br/cel/aplicacao/add_usuario.php?cadastrado=&novo=true&login=requisitosBrainly'

In [7]:
def forceLogin(url):
    result = session_requests.post(url)
    
    return result

In [8]:
result = forceLogin(PAYLOAD)

In [9]:
def treat_symbols_id(table_data):
    #link_symbol = re.compile('(<a.+(id=\d{0,5}).+>(\w*)</a>)')
    #link_symbol = re.compile('(<a.+(id=\d{0,5}).*">)([^<]*).*/a>')
    #a_tag = re.compile(r'(<a[^>]*>[^<]+<\/a>)')
    a_tag = re.compile(r'(<a[\s]+[^>]+>(?:.(?!\<\/a\>))*.<\/a>)')
    link_symbol = re.compile(r'(<a.+(id=\d{0,5}).*">([^<]*).*/a>)')
    
    
    tables = []
    for table in table_data:
        rows = []
        for row in table:            
            tags = re.findall(a_tag, r'{}'.format(row))
            
            for tag in tags:                
                symbol = re.findall(link_symbol, str(tag))
                
                link, id_symbol, name = symbol[0]
                
                
                row = str(row)
                row = row.replace(r'{}'.format(link), r'[{}](#{})'.format(name, id_symbol))
            
            row = BeautifulSoup(str(row))
            row = row.findAll('td')
            
            rows.append(row)
        tables.append(rows)
        
    return tables

In [10]:
def pull_data(data_url):
    data = session_requests.get(data_url)
    soup = BeautifulSoup(data.content)
    
    tables = soup.findAll('table')
    
    del tables[1]
    
    table_data = []
    for table in tables:
        data = table.findAll('td')
        table_data.append(data)
                
    table_data = treat_symbols_id(table_data)

    info_data = table_data[0]
    related_data = table_data[1]

    return table_data

In [11]:
def lexicos_cenarios_objects(id_lexico, info, related_data, type_obj):
    final_data = []
    
    final_data.append(int(id_lexico))
    for data in info:
        data = data[0].text
        data = data.replace('', '').replace('\r', '').replace('\t', '')
        data = data.strip(' ').strip('\n').strip(' ')

        final_data.append(data)
    
    related_lexicos = []
    for related in related_data:
        data = related[0].text
        if (len(data) > 0):
            related_lexicos.append(str(data))
            #related_lexicos += data + '\n'

    final_data.append(related_lexicos)
    
    if type_obj == 'lexicos':
        lex = Lexico(*final_data)
        return lex
    elif type_obj == 'cenarios':
        cen = Cenario(*final_data)
        return cen
    return final_data

In [12]:
def lexicos_information(urls_lexicos):
    lexicos = []
    for url in urls_lexicos:
        info, related_data = pull_data(url[0])
        id_lex = int(url[1])
        obj = lexicos_cenarios_objects(id_lex, info, related_data, 'lexicos')

        lexicos.append(obj)
    
    return lexicos

In [13]:
LEXICOS = lexicos_information(urls_lexicos)

# Write to markdown file

## Léxicos

In [14]:
def analyse_id(relacionados):
    id_regex = re.compile('id=(\d{0,5})')
    
    lexicos = []
    cenarios = []
    for relacionado in relacionados:
        relacionado = relacionado.replace('\n', '</br>')
        id = re.findall(id_regex, relacionado)
        id = int(id[0])

        if (id < 8000):
            cenarios.append(relacionado)
        else:
            lexicos.append(relacionado)
    
    
    while len(cenarios) > len(lexicos):
        lexicos.append(' ')
        
    while len(lexicos) > len(cenarios):
        cenarios.append(' ')
            
    return lexicos, cenarios

In [15]:
def generate_lexico_markdown(LEXICOS):
    hash = hex(random.getrandbits(128))

    f = open('./testes/lexicos{}.md'.format(hash), "w+")

    for lexico in LEXICOS:
        f.write("# %s {#id=%d}\n\n" % (lexico.nome, lexico.id_lex))

        nome = lexico.nome.replace('\n', '</br>')
        nocao = lexico.nocao.replace('\n', '</br>')
        classificacao = lexico.classificacao.replace('\n', '</br>')
        impacto = lexico.impacto.replace('\n', '</br>')
        sinonimo = lexico.sinonimo.replace('\n', '</br>')

        f.write('|  Informações  | Símbolo |\n')
        f.write('|:-------------:|:-------:|\n')
        f.write('|      Nome     |    {}   |\n'.format(nome))
        f.write('|      Noção    |    {}   |\n'.format(nocao))    
        f.write('| Classificação |    {}   |\n'.format(classificacao))    
        f.write('|    Impacto    |    {}   |\n'.format(impacto))
        f.write('|    Sinônimos  |    {}   |\n'.format(sinonimo))

        f.write('\n\n')

        f.write('|    Cenários   | Lexicos |\n')
        f.write('|:-------------:|:-------:|\n')

        lexicos, cenarios = analyse_id(lexico.relacionados)

        if len(lexicos) > 0:
            for i in range(len(lexicos)):
                f.write('| {} | {} |\n'.format(cenarios[i], lexicos[i]))
        else:
            f.write('| - | - |\n')

        f.write('\n\n\n')

    f.close()

In [22]:
generate_lexico_markdown(LEXICOS)

In [17]:
analyse_id(LEXICOS[3].relacionados)

([' '], ['[moderar denúncia](#id=3030)'])

## Cenários

In [18]:
def cenarios_information(urls_cenarios):
    cenarios = []
    for url in urls_cenarios:
        info, related_data = pull_data(url[0])
        id_cen = int(url[1])
        obj = lexicos_cenarios_objects(id_cen, info, related_data, 'cenarios')

        cenarios.append(obj)
    
    return cenarios

In [19]:
CENARIOS = cenarios_information(urls_cenarios)

In [20]:
def generate_cenario_markdown(CENARIO):
    hash = hex(random.getrandbits(128))

    f = open('./testes/cenarios{}.md'.format(hash), "w+")

    for cenario in CENARIOS:
        f.write("# %s {#id=%d}\n\n" % (cenario.titulo, cenario.id_cen))

        titulo = cenario.titulo.replace('\n', '</br>')
        objetivo = cenario.objetivo.replace('\n', '</br>')
        contexto = cenario.contexto.replace('\n', '</br>')
        atores = cenario.atores.replace('\n', '</br>')
        recursos = cenario.recursos.replace('\n', '</br>')
        excecao = cenario.excecao.replace('\n', '</br>')
        episodios = cenario.episodios.replace('\n', '</br>')

        f.write('|  Informações  | Cenário |\n')
        f.write('|:-------------:|:-------:|\n')
        f.write('|    Título     |    {}   |\n'.format(titulo))
        f.write('|    Objetivo   |    {}   |\n'.format(objetivo))    
        f.write('|    Contexto   |    {}   |\n'.format(contexto))    
        f.write('|    Atores     |    {}   |\n'.format(atores))
        f.write('|    Recursos   |    {}   |\n'.format(recursos))
        f.write('|    Excecão    |    {}   |\n'.format(excecao))
        f.write('|    Episódio   |    {}   |\n'.format(str(episodios)))

        f.write('\n\n')

        f.write('|    Cenários   |\n')
        f.write('|:-------------:|\n')

        if len(cenario.cen_relacionados) > 0:
            for i in range(len(cenario.cen_relacionados)):
                f.write('| {} |'.format(cenario.cen_relacionados[i]))
        else:
            f.write('| - | - |\n')

        f.write('\n\n\n')

    f.close()

In [23]:
generate_cenario_markdown(CENARIOS)

In [ ]:
print(CENARIOS[2].episodios)